In [ ]:
!pip install uvicorn==0.34.2
!pip install fastapi==0.115.12
!pip install pandas==2.2.2
!pip install numpy==2.0.2
!pip install pydantic==2.11.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/MLProject

/content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/MLProject


In [ ]:
# fast api 서버 오픈용 라이브러리
import uvicorn
import joblib
# fast api 라이브러리
from fastapi import FastAPI
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# 타 인터페이스 언어와 연계하기 위한 데이터 관리 라이브러리
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

# 1. CORS 미들웨어 추가

In [ ]:
app = FastAPI(title="ML API")
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=['*']
)

In [ ]:
# 2. 모델 불러오기
loaded = joblib.load('mlcore.dump')
loadedModel = loaded['model']
print(loaded)
features = loaded['features']
scaler = loaded['preprocessing_feature'][0]
print(scaler)

{'model': SVC(), 'features': ['AGE', 'ISACTIVEMEMBER', 'HBLA', 'SPUSER'], 'label': ['EXITED'], 'preprocessing_feature': [MinMaxScaler()]}
MinMaxScaler()


In [ ]:
inputAge = 30
inputIsactivemember = 1
inputHbla = 0
inputSpuser = 0

inDf = pd.DataFrame([[inputAge,inputIsactivemember, inputHbla, inputSpuser]])
inDf
result = loadedModel.predict(inDf)[0]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
class InDataSet(BaseModel):
    AGE : int
    ISACTIVEMEMBER : int
    HBLA : int
    SPUSER : int

# 4. 예측용 함수 정의

In [ ]:
@app.post("/predictBase", status_code=200)
async def predict_tf(x: InDataSet):
    inputAge = x.inputAge
    inputIsactivemember = x.inputIsactivemember
    inputHbla = x.inputHbla
    inputSpuser = x.inputSpuser

    # 분석 모델 로직
    inDf = pd.DataFrame([[inputAge,inputIsactivemember, inputHbla, inputSpuser]])
    inDf
    result = loadedModel.predict(inDf)[0]
    return {"이탈여부": result}


In [ ]:
@app.get("/")
async def root():
    return {"message": "online"}

# 5. 코랩에서 오픈시 (colab & ngrok용)

In [ ]:
!pip install  nest-asyncio pyngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn

auth_token = ""
ngrok.set_auth_token(auth_token)
ngrokTunnel = ngrok.connect(9999)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=9999)

INFO:     Started server process [204]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:9999 (Press CTRL+C to quit)


공용 URL https://f559-34-91-40-25.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [204]


# 5. 서버 오픈 (일반 서버용 : AWS)

In [ ]:
import uvicorn